## Работа с демо данными Яндекс.Метрики

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Загрузим данные и посмотрим на первые 5 строк датафрейма
df = pd.read_csv('yandex_metrics.csv')
df.head()

,date,browser,user_type,visits
0,2020-06-15,Google Chrome,People,250.0
1,2020-06-15,Google Chrome,Robots,104.0
2,2020-06-15,Yandex Browser,People,47.0
3,2020-06-15,Firefox,Robots,31.0
4,2020-06-15,Chrome Mobile,People,31.0


## Задания:

#### Давайте выясним, с какого браузера приходит больше всего ботов. Посмотрите на данные и определите, у какого браузера самая большая доля посещений сайта ботами от общего числа посещений в этом браузере?

In [3]:
bots = df \
         .groupby(['browser', 'user_type'], as_index=False) \
         .agg({'visits': 'sum'}) \
         .pivot(index='browser', columns='user_type', values='visits') \
         .fillna(0) \
         .reset_index()

bots['bot_percentage'] = round(bots.Robots/(bots.People + bots.Robots), 2)
bots

user_type,browser,People,Robots,bot_percentage
0,Android Browser,1.0,0.0,0.00
1,Chrome Mobile,31.0,0.0,0.00
2,Chromium,1.0,1.0,0.50
3,CocCoc,1.0,0.0,0.00
4,Edge,4.0,1.0,0.20
5,Firefox,27.0,31.0,0.53
6,Google Chrome,250.0,104.0,0.29
7,Mobile Safari,15.0,1.0,0.06
8,Opera,11.0,5.0,0.31
9,Safari,19.0,22.0,0.54


In [4]:
bots_loved = bots.loc[bots.bot_percentage.idxmax()]
print(f'Больше всего ботов приходит с браузера - {bots_loved.browser}, {bots_loved.bot_percentage:.0%} посещений.')

Больше всего ботов приходит с браузера - UCWEB, 100% посещений.


#### Создайте колонку с метрикой доли заходов на сайт для каждого браузера. В ответе укажите процент открытий сайта настоящими пользователями Safari (не ботами) с точностью до 1 знака после точки.

In [5]:
openings = df \
         .groupby(['browser', 'user_type'], as_index=False) \
         .agg({'visits': 'sum'})

openings['percentage'] = round(openings.visits/openings.visits.sum() * 100, 1)
openings.loc[(openings.browser == 'Safari') & (openings.user_type == 'People')]

,browser,user_type,visits,percentage
15,Safari,People,19.0,3.2


#### Другой аналитик уже агрегировал часть данных из метрики. Вам нужно объединить усилия, чтобы получить финальный отчет. Прочитайте дополнительные данные из Google SpreadSheet, объедините вашу таблицу из предыдущего шага с таблицей аналитика и посчитайте количество визитов на человека в разрезе по браузеру и значениям Robots/People.

#### В ответе укажите, сколько визитов на уникального пользователя совершается ботами с браузера Google Chrome (округлите до целого).

In [6]:
from io import BytesIO
import requests
r = requests.get('https://docs.google.com/spreadsheets/d/e/2PACX-1vQWMsvBTVio9C7IOOxfFO9C15BRHyME-_ENHqBodDOjuiHwk9fCuF5hUVmDs497PZOqPYK3exdSikOK/pub?gid=1006633900&single=true&output=csv')
data = r.content

df_upd = pd.read_csv(BytesIO(data))

In [7]:
full_df = pd.merge(df, df_upd, on=['date', 'browser', 'user_type'], how='inner')
full_df.head()

,date,browser,user_type,visits,users
0,2020-06-15,Google Chrome,People,250.0,240.0
1,2020-06-15,Google Chrome,Robots,104.0,104.0
2,2020-06-15,Yandex Browser,People,47.0,44.0
3,2020-06-15,Firefox,Robots,31.0,29.0
4,2020-06-15,Chrome Mobile,People,31.0,30.0


In [8]:
full_df['bot_per_user'] = full_df.visits/full_df.users
full_df.query('browser == "Google Chrome" & user_type == "Robots"')

,date,browser,user_type,visits,users,bot_per_user
1,2020-06-15,Google Chrome,Robots,104.0,104.0,1.0
